In [1]:
import pandas as pd
import numpy as np

from requests import get
import re
from bs4 import BeautifulSoup
import os

### Codeup Blogs

**Goals:** Write a function to scrape urls from main Codeup blog web page and write a function that returns a dictionary of blog titles and text for each blog page. 

#### Grab Title from Page

Here I use the `.find()` method on my soup with the `<h1>` tag. As always, there is no one way to accomplish our task, so I'm demonstrating one way to scrape the headline, not THE way to scrape the headline. 

In [2]:
url = 'https://codeup.com/codeups-data-science-career-accelerator-is-here/'
headers = {'User-Agent': 'Codeup Data Science'} 
    
response = get(url, headers=headers)
response.ok

True

In [3]:
# Here's our long string of HTML; we'll use response.text to make our soup object.

print(type(response.text))

<class 'str'>


In [4]:
# Create our Soup object by passing our HTML string and choice of parser.

soup = BeautifulSoup(response.text, 'html.parser')

# Now we have our BeautifulSoup object and can use its built-in methods and attributes.

print(type(soup))

<class 'bs4.BeautifulSoup'>


In [5]:
# The h1 element holds my title.

title = soup.find('h1').text
title

'Codeup’s Data Science Career Accelerator is Here!'

#### Grab Text from Page

In [6]:
content = soup.find('div', class_="jupiterx-post-content").text
print(content)

The rumors are true! The time has arrived. Codeup has officially opened applications to our new Data Science career accelerator, with only 25 seats available! This immersive program is one of a kind in San Antonio, and will help you land a job in Glassdoor’s #1 Best Job in America.
Data Science is a method of providing actionable intelligence from data. The data revolution has hit San Antonio, resulting in an explosion in Data Scientist positions across companies like USAA, Accenture, Booz Allen Hamilton, and HEB. We’ve even seen UTSA invest $70 M for a Cybersecurity Center and School of Data Science. We built a program to specifically meet the growing demands of this industry.
Our program will be 18 weeks long, full-time, hands-on, and project-based. Our curriculum development and instruction is led by Senior Data Scientist, Maggie Giust, who has worked at HEB, Capital Group, and Rackspace, along with input from dozens of practitioners and hiring partners. Students will work with real

In [7]:
print(type(content))

<class 'str'>


#### Build Blog Function

In [8]:
def get_blog_articles(urls, cached=True):
    '''
    This function takes in a list of Codeup Blog urls and a parameter
    with default cache == False which returns a df from a csv file.
    If cache == True, the function scrapes the title and text for each url, 
    creates a list of dictionaries with the title and text for each blog, 
    converts list to df, and returns df.
    '''
    if cached == True:
        df = pd.read_json('big_blogs.json')
    else:
        headers = {'User-Agent': 'Codeup Bayes Data Science'} 

        # Create an empty list to hold dictionaries
        articles = []

        # Loop through each url in our list of urls
        for url in urls:

            # get request to each url saved in response
            response = get(url, headers=headers)

            # Create soup object from response text and parse
            soup = BeautifulSoup(response.text, 'html.parser')

            # Save the title of each blog in variable title
            title = soup.find('h1').text

            # Save the text in each blog to variable text
            content = soup.find('div', class_="jupiterx-post-content").text

            # Create a dictionary holding the title and content for each blog
            article = {'title': title, 'content': content}

            # Add each dictionary to the articles list of dictionaries
            articles.append(article)
            
        # convert our list of dictionaries to a df
        df = pd.DataFrame(articles)

        # Write df to a json file for faster access
        df.to_json('big_blogs.json')
    
    return df

#### Test Function

In [14]:
# Here cached == False, so the function will do a fresh scrape of the urls and write data to a json file.

urls = ['https://codeup.com/codeups-data-science-career-accelerator-is-here/',
        'https://codeup.com/data-science-myths/',
        'https://codeup.com/data-science-vs-data-analytics-whats-the-difference/',
        'https://codeup.com/10-tips-to-crush-it-at-the-sa-tech-job-fair/',
        'https://codeup.com/competitor-bootcamps-are-closing-is-the-model-in-danger/']

blogs = get_blog_articles(urls=urls, cached=False)
blogs

,title,content
0,Codeup’s Data Science Career Accelerator is Here!,The rumors are true! The time has arrived. Cod...
1,Data Science Myths,By Dimitri Antoniou and Maggie Giust\nData Sci...
2,Data Science VS Data Analytics: What’s The Dif...,"By Dimitri Antoniou\nA week ago, Codeup launch..."
3,10 Tips to Crush It at the SA Tech Job Fair,SA Tech Job Fair\nThe third bi-annual San Anto...
4,Competitor Bootcamps Are Closing. Is the Model...,Competitor Bootcamps Are Closing. Is the Model...


#### Bonus URL Scrape

In [15]:
# I'm going to hit Codeup's main blog page to scrape the urls

url = 'https://codeup.com/resources/#blog'
headers = {'User-Agent': 'Codeup Data Science'} 

# Request the HTML
response = get(url, headers=headers)

# Create the soup object to parse the HTML
soup = BeautifulSoup(response.text, 'html.parser')

In [29]:
# I'm filtering my soup to return a list of all anchor elements from my HTML.

urls_list = soup.find_all('a', class_='jet-listing-dynamic-link__link')
urls_list

[<a class="jet-listing-dynamic-link__link" href="https://codeup.com/introducing-salary-refund-guarantee/"><span class="jet-listing-dynamic-link__label">Introducing Our Salary Refund Guarantee</span></a>,
 <a class="jet-listing-dynamic-link__link" href="https://codeup.com/introducing-salary-refund-guarantee/"><span class="jet-listing-dynamic-link__label">Read More</span></a>,
 <a class="jet-listing-dynamic-link__link" href="https://codeup.com/new-scholarship/"><span class="jet-listing-dynamic-link__label">Announcing: The Annie Easley Scholarship to Support the Black Community in Tech</span></a>,
 <a class="jet-listing-dynamic-link__link" href="https://codeup.com/new-scholarship/"><span class="jet-listing-dynamic-link__label">Read More</span></a>,
 <a class="jet-listing-dynamic-link__link" href="https://codeup.com/codeup-in-houston/"><span class="jet-listing-dynamic-link__label">Codeup Launches Houston!</span></a>,
 <a class="jet-listing-dynamic-link__link" href="https://codeup.com/codeu

In [33]:
# Filter the href attribute value for each anchor element in my list; we scraped 40 urls.

# I'm using a set comprehension to return only unique urls because there are two links for each article.
urls = {link.get('href') for link in urls_list}

# I'm converting my set to a list of urls.
urls = list(urls)

print(f'There are {len(urls)} unique links in our urls list.')
print()
urls 

There are 20 unique links in our urls list.



['https://codeup.com/build-your-career-in-tech/',
 'https://codeup.com/introducing-salary-refund-guarantee/',
 'https://codeup.com/what-is-machine-learning/',
 'https://codeup.com/what-is-python/',
 'https://codeup.com/what-to-expect-at-codeup/',
 'https://codeup.com/transition-into-data-science/',
 'https://codeup.com/education-is-an-investment/',
 'https://codeup.com/what-data-science-career-is-for-you/',
 'https://codeup.com/codeups-application-process/',
 'https://codeup.com/codeup-in-houston/',
 'https://codeup.com/succeed-in-a-coding-bootcamp/',
 'https://codeup.com/covid-19-data-challenge/',
 'https://codeup.com/journey-into-web-development/',
 'https://codeup.com/codeup-wins-civtech-datathon/',
 'https://codeup.com/how-were-celebrating-world-mental-health-day-from-home/',
 'https://codeup.com/codeup-alumni-make-water/',
 'https://codeup.com/codeup-inc-5000/',
 'https://codeup.com/new-scholarship/',
 'https://codeup.com/from-slacker-to-data-scientist/',
 'https://codeup.com/math

#### Bonus URL Function

In [34]:
def get_all_urls():
    '''
    This function scrapes all of the Codeup blog urls from
    the main Codeup blog page and returns a list of urls.
    '''
    # The base url for the main Codeup blog page
    url = 'https://codeup.com/resources/#blog'
    headers = {'User-Agent': 'Codeup Data Science'} 
    
    # Send request to main page and get response.
    response = get(url, headers=headers)
    
    # Create soup object using response.
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Create a list of the anchor elements that hold the urls.
    urls_list = soup.find_all('a', class_='jet-listing-dynamic-link__link')
    
    # I'm using a set comprehension to return only unique urls because there are two links for each article.
    urls = {link.get('href') for link in urls_list}

    # I'm converting my set to a list of urls.
    urls = list(urls)
        
    return urls

In [35]:
# Now I can use my same function with my new function.
# cached == False does a fresh scrape.

big_blogs = get_blog_articles(urls=get_all_urls(), cached=False)

In [36]:
big_blogs.head(10)

,title,content
0,Build Your Career in Tech: Advice from Alumni!,"Bryan Walsh, Codeup Web Development alum, and ..."
1,Introducing Our Salary Refund Guarantee,"Here at Codeup, we believe it’s time to revolu..."
2,What is Machine Learning?,"There’s a lot we can learn about machines, and..."
3,What is Python?,If you’ve been digging around our website or r...
4,What to Expect at Codeup,"Setting Expectations for Life Before, During, ..."
5,What is the Transition into Data Science Like?,Alumni Katy Salts and Brandi Reger joined us a...
6,Your Education is an Investment,You have many options regarding educational ro...
7,What Data Science Career is For You?,If you’re struggling to see yourself as a data...
8,What is Codeup’s Application Process?,Curious about Codeup’s application process? Wo...
9,Codeup Launches Houston!,"Houston, we have a problem: there aren’t enoug..."


In [ ]:
big_blogs.info()

In [37]:
# cached == True reads in a df from `big_blogs.json`.

big_blogs = get_blog_articles(urls=get_all_urls(), cached=True)
big_blogs.head()

,title,content
0,Build Your Career in Tech: Advice from Alumni!,"Bryan Walsh, Codeup Web Development alum, and ..."
1,Introducing Our Salary Refund Guarantee,"Here at Codeup, we believe it’s time to revolu..."
2,What is Machine Learning?,"There’s a lot we can learn about machines, and..."
3,What is Python?,If you’ve been digging around our website or r...
4,What to Expect at Codeup,"Setting Expectations for Life Before, During, ..."


___

### Inshorts News Articles

**Goal:**  Write a function that scrapes the news articles for the following topics:

- Business


- Sports


- Technology


- Entertainment

In [39]:
url = 'https://inshorts.com/en/read/entertainment'

response = get(url)
response.ok

True

In [40]:
soup = BeautifulSoup(response.text, 'html.parser')

#### Scrape News Cards from Main Page

In [49]:
# Scrape a ResultSet of all the news cards on the page and inspect the elements on the first card.

cards = soup.find_all('div', class_='news-card')

print(f'There are {len(cards)} news cards on this page.')
print()
cards[0]

There are 25 news cards on this page.



<div class="news-card z-depth-1" itemscope="" itemtype="http://schema.org/NewsArticle">
<span content="" itemid="https://inshorts.com/en/news/sushant-singh-rajputs-cousin-neeraj-leading-from-chhatapur-in-bihar-polls-1605013190280" itemprop="mainEntityOfPage" itemscope="" itemtype="https://schema.org/WebPage"></span>
<span itemprop="author" itemscope="itemscope" itemtype="https://schema.org/Person">
<span content="Arshiya Chopra" itemprop="name"></span>
</span>
<span content="Sushant Singh Rajput's cousin Neeraj leading from Chhatapur in Bihar polls" itemprop="description"></span>
<span itemprop="image" itemscope="" itemtype="https://schema.org/ImageObject">
<meta content="https://static.inshorts.com/inshorts/images/v1/variants/jpg/m/2020/11_nov/10_tue/img_1605011797271_95.jpg?" itemprop="url"/>
<meta content="864" itemprop="width"/>
<meta content="483" itemprop="height"/>
</span>
<span itemprop="publisher" itemscope="itemscope" itemtype="https://schema.org/Organization">
<span content=

#### Scrape the Title from Each News Card

In [51]:
# Create a list of titles using the span element and itemprop attribute with text method.

titles = [card.find('span', itemprop='headline').text for card in cards]
titles[:5]

["Sushant Singh Rajput's cousin Neeraj leading from Chhatapur in Bihar polls",
 "I'm out of 'Prison Break', done with playing straight characters: Miller",
 "Deepika Padukone participates in FrontRow's $3.2 mn fundraise",
 "Anushka shares pic of Kohli cleaning his 'muddy spikes' ahead of Australia tour",
 'Ranveer climbs onto the roof of his car, gestures fans to keep sanitising hands']

#### Scrape Author from News Cards

In [52]:
# Create a list of authors using the span element and class attribute with text method.

authors = [card.find('span', class_='author').text for card in cards]
authors[:5]

['Arshiya Chopra',
 'Pragya Swastik',
 'Rishabh Bhatnagar',
 'Anmol Sharma',
 'Pragya Swastik']

#### Scrape Text from News Cards

In [53]:
# Create a list of content strings using the div element and itemprop attribute with text method.

content = [card.find('div', itemprop='articleBody').text for card in cards]
content[:5]

["Late actor Sushant Singh Rajput's cousin and BJP leader Neeraj Kumar Singh is leading from Chhatapur seat in Bihar Assembly elections. He is ahead with a margin of over 22,000 votes against Rashtriya Janata Dal (RJD) candidate Vipin Kumar Singh. Neeraj had won from the seat in the last Assembly election on the BJP ticket.",
 'Actor Wentworth Miller, who played Michael Scofield in the series \'Prison Break\', wrote on Instagram, "I\'m out. Of PB. Officially." Miller, who came out as gay in 2013, added, "I don\'t want to play straight characters. Their stories have been told." He also said, "If you were a fan of the show, hoping for additional seasons...I understand this is disappointing."',
 'Creative arts and sports-focused learning platform FrontRow has raised $3.2 million in seed funding led by Lightspeed and Elevation Capital (formerly SAIF Partners), along with Deepika Padukone’s family office. Founded earlier this year by Mikhil Raj, Shubhadit Sharma and Ishaan Preet Singh, the 

In [54]:
# Create an empty list, articles, to hold the dictionaries for each article.
articles = []

# Loop through each news card on the page and get what we want
for card in cards:
    title = card.find('span', itemprop='headline' ).text
    author = card.find('span', class_='author').text
    content = card.find('div', itemprop='articleBody').text
    
    # Create a dictionary, article, for each news card
    article = {'title': title, 'author': author, 'content': content}
    
    # Add the dictionary, article, to our list of dictionaries, articles.
    articles.append(article)

In [56]:
# Here we see our list contains 24-25 dictionaries for news cards

print(len(articles))
articles[0]

25


{'title': "Sushant Singh Rajput's cousin Neeraj leading from Chhatapur in Bihar polls",
 'author': 'Arshiya Chopra',
 'content': "Late actor Sushant Singh Rajput's cousin and BJP leader Neeraj Kumar Singh is leading from Chhatapur seat in Bihar Assembly elections. He is ahead with a margin of over 22,000 votes against Rashtriya Janata Dal (RJD) candidate Vipin Kumar Singh. Neeraj had won from the seat in the last Assembly election on the BJP ticket."}

#### Build Article Function

In [57]:
def get_news_articles(cached=False):
    '''
    This function uses a cache parameter with default cache == False to give the option of 
    returning in a df of inshorts topics and info by reading a csv file or
    of doing a fresh scrape of inshort pages with topics business, sports, technology,
    and entertainment and writing the returned df to a csv file.
    '''
    # option to read in a json file instead of scrape for df
    if cached == True:
        df = pd.read_json('articles.json')
        
    # cached == True completes a fresh scrape for df    
    else:
    
        # Set base_url and headers that will be used in get request

        base_url = 'https://inshorts.com/en/read/'
        headers = {'User-Agent': 'Codeup Data Science'}
        
        # List of topics to scrape
        topics = ['business', 'sports', 'technology', 'entertainment']

        # Create an empty list, articles, to hold our dictionaries
        articles = []

        for topic in topics:

            # Get a response object from the main inshorts page
            response = get(base_url + topic, headers=headers)

            # Create soup object using response from inshort
            soup = BeautifulSoup(response.text, 'html.parser')

            # Scrape a ResultSet of all the news cards on the page
            cards = soup.find_all('div', class_='news-card')

            # Loop through each news card on the page and get what we want
            for card in cards:
                title = card.find('span', itemprop='headline' ).text
                author = card.find('span', class_='author').text
                content = card.find('div', itemprop='articleBody').text

                # Create a dictionary, article, for each news card
                article = ({'topic': topic, 
                            'title': title, 
                            'author': author, 
                            'content': content})

                # Add the dictionary, article, to our list of dictionaries, articles.
                articles.append(article)
            
        # Create a DataFrame from list of dictionaries
        df = pd.DataFrame(articles)
        
        # Write df to csv for future use
        df.to_json('articles.json')
    
    return df

In [58]:
# Test our function with cached == False to do a freash scrape and create `articles.json` file.

df = get_news_articles(cached=False)
df.head()

,topic,title,author,content
0,business,"Sensex ends at record high of 42,597, Nifty en...",Pragya Swastik,Sensex and Nifty ended at record highs on Mond...
1,business,China COVID-19 vaccine trial halted in Brazil ...,Ankush Verma,The final-stage trial of China's Sinovac COVID...
2,business,Likely the most significant medical advance in...,Krishna Veera Vanamali,After Pfizer announced its experimental COVID-...
3,business,"Gold prices fall below ₹50,000 per 10g on Pfiz...",Krishna Veera Vanamali,"Gold prices in India fell by ₹2,502 to ₹49,665..."
4,business,Pfizer's coronavirus vaccine needs to be store...,Krishna Veera Vanamali,"Pfizer's COVID-19 vaccine, which used syntheti..."


In [59]:
df.topic.value_counts()

technology       25
sports           25
entertainment    25
business         24
Name: topic, dtype: int64

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   topic    99 non-null     object
 1   title    99 non-null     object
 2   author   99 non-null     object
 3   content  99 non-null     object
dtypes: object(4)
memory usage: 3.2+ KB


In [ ]:
# Test our function to read in the df from `articles.csv`

df = get_news_articles(cached=True)
df.head()

In [ ]:
df.info()